# 1. Подготовка

In [1]:
import pandas as pd
import numpy as np
import torch
import transformers
import string 
from spacy.lang.en import English
from nltk.corpus import stopwords as stopwords
from tqdm import notebook
from tqdm import tqdm

In [8]:
df=pd.read_csv(r'C:\Users\Andrey\Desktop\Soft balls\Yandex\WorkBooks\toxic_comments.csv')

In [9]:
df['toxic'].value_counts(normalize=True)

0    0.898321
1    0.101679
Name: toxic, dtype: float64

Классы не сбалансированны.

In [5]:
tqdm.pandas(desc="my bar!") # для отображения временных затрат на работу ячеек

C:\Users\Andrey\Anaconda3\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# 2. Обучение

Для подготовки данных воспользуемся библиотекой SPACY, произведем токенизацию посредствам данной библиотек.

## spacy

In [6]:
import spacy

In [7]:
nlp = spacy.load("en_core_web_sm") #загрузим модель spacy

In [8]:
from spacy.lang.en.stop_words import STOP_WORDS
import string 
from spacy.lang.en import English
from nltk.corpus import stopwords as stopwords
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

In [9]:
punctuations=string.punctuation
punctuations+=('•')

In [10]:
punctuations

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~•'

выберем стоп-слова из библиотек spacy

In [11]:
stop_words_spacy=STOP_WORDS 

In [12]:
def tokenization(row):
    tokens=nlp(row) # произведем токенизацию строки
    tokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens ] #приведем все слова к нижнему регистру 
    tokens = [ word for word in tokens if word not in stop_words_spacy and word not in punctuations ] # удалим знаки препинания
    tokens=' '.join(tokens) # Объединим списки в str
    return tokens

In [13]:
lemma=df['text'].progress_apply(tokenization)

my bar!: 100%|█████████████████████████████████████████████████████████████████| 159571/159571 [41:16<00:00, 64.42it/s]


In [14]:
df['lemma']=lemma.copy()

In [15]:
df['lemma']

0         explanation edit username hardcore metallica f...
1         d'aww match background colour seemingly stick ...
2         hey man try edit war guy constantly remove rel...
3         real suggestion improvement wonder section sta...
4                             sir hero chance remember page
                                ...                        
159566    second time asking view completely contradict ...
159567        ashamed horrible thing talk page 128.61.19.93
159568    spitzer umm s actual article prostitution ring...
159569        look like actually speedy version delete look
159570    ... think understand come idea bad right away ...
Name: lemma, Length: 159571, dtype: object

### Вывод

- мы изучили данные
- провели лемматизацию наших данных

## Обучение SPACY

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn import metrics
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.pipeline import make_pipeline

Раздели выборки на обучающую и тестовую. На обучающей проведем кросс-валидацию.

In [17]:
X = df['lemma']
y = df['toxic'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=42)

Воспользуемся pipeline для того, что бы провести кросс валидацию.

In [21]:
def result_calculation(X,y,balanced=True):
    results={}
    for model,model_name in zip((LogisticRegression,LGBMClassifier,RandomForestClassifier),
                                ('LogisticRegression','LGBMClassifier','RandomForestClassifier')):
        if balanced==True:
            classifier =model(class_weight='balanced')
        elif balanced==False:
            classifier =model()
        clf = make_pipeline(TfidfVectorizer(), classifier)
        scores = cross_val_score(clf, X, y, scoring='f1', cv=5)
        results[model_name]=np.mean(scores)
        print(results)
    return results

In [22]:
tf_idf_metod=result_calculation(X_train,y_train)

{'LogisticRegression': 0.7497422751382493}
{'LogisticRegression': 0.7497422751382493, 'LGBMClassifier': 0.7352967725263264}
{'LogisticRegression': 0.7497422751382493, 'LGBMClassifier': 0.7352967725263264, 'RandomForestClassifier': 0.6469286160449809}


In [23]:
tf_idf_metod

{'LogisticRegression': 0.7497422751382493,
 'LGBMClassifier': 0.7352967725263264,
 'RandomForestClassifier': 0.6469286160449809}

In [24]:
pd.DataFrame(tf_idf_metod,index=['F1'])

,LogisticRegression,LGBMClassifier,RandomForestClassifier
F1,0.749742,0.735297,0.646929


### Вывод

Мы провели кросс-валидацию, по ее рузльтатам лушей моделью оказалась LogisticRegression.   
Ее рузультат 0.75

## Test

In [25]:
classifier =LogisticRegression(class_weight='balanced')

In [26]:
clf = make_pipeline(TfidfVectorizer(), classifier)
clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token...ern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('logisticregression',
                 LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                                    fit_inte

In [27]:
predicted=clf.predict(X_test)

In [28]:
predicted

array([1, 0, 0, ..., 1, 0, 0], dtype=int64)

In [29]:
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))
print("Logistic Regression F1:",metrics.f1_score(y_test, predicted))

Logistic Regression Accuracy: 0.942889371657754
Logistic Regression Precision: 0.6741224489795918
Logistic Regression Recall: 0.8483665502362852
Logistic Regression F1: 0.7512736535662299


### Вывод

Мы провели тестирование подобранной модели LogisticRegression на тестовой выборке.  
Результат превысил целевой.

## BERT

Применем облегченную версию BERT -DistilBERT уже предобученную.

In [30]:
model = transformers.DistilBertModel.from_pretrained('distilbert-base-uncased')

In [31]:
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

## Обучение BERT

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn import metrics
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.utils import shuffle

In [13]:
def BERT(data):
    # из полученных данных на этапе токенизации выделим тензоры ids и attention mask
    attention_mask=[]
    padded=[]
    for i in data:
        padded.append(i['input_ids'])
        attention_mask.append(i['attention_mask'])
    # выберем батчи равные 50 для рассчетов эмбендингов
    batch_size =50
    embeddings = []
    for i in notebook.tqdm(range(np.array(data).shape[0] // batch_size)):
            batch=torch.cat(padded[batch_size*i:batch_size*(i+1)], dim=0) # обединим списки тензоров в список тензоров ids
            attention_mask_batch=torch.cat(attention_mask[batch_size*i:batch_size*(i+1)], dim=0) # обединим списки тензоров в список тензоров attention masks


            with torch.no_grad():
                batch_embeddings = model(batch, attention_mask=attention_mask_batch)

            embeddings.append(batch_embeddings[0][:,0,:].numpy())
    return np.concatenate(embeddings)

In [14]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    features_downsampled = pd.concat([features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat([target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    features_downsampled, target_downsampled = shuffle(
    features_downsampled, target_downsampled, random_state=12345)
    return features_downsampled, target_downsampled

Т.к. наши классы несбалансированны, то применем метод даунсемплинг для его балансировки

In [15]:
features, targets= downsample(df['text'],df['toxic'],0.11) 

In [16]:
targets.value_counts(normalize=True)

1    0.507142
0    0.492858
Name: toxic, dtype: float64

In [37]:
features['tokenized'] = features.progress_apply(
    lambda x: tokenizer.encode_plus(
  x,
  add_special_tokens=True, # Add '[CLS]' and '[SEP]'
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      max_length=50, 
        # данное число выбрано для ускорения обучения модели, целом если после токенизации считать 0.75 квантиль равен 100. 
        # Правильнее применить его, но тогда неполучается все рассчиать 
      return_tensors='pt',truncation=True
        # Return PyTorch tensors
))
# на выходе получаем два словаря с ids тензорами и attention mask тензорами. 
# В дальнейшем разделим их на списки и будем объединят под каждую батчу

my bar!: 100%|██████████████████████████████████████████████████████████████████| 31993/31993 [00:41<00:00, 776.48it/s]


Разделим уже токенизированную выборку на тестовую и обучающую

In [18]:
X = features
y = targets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=42)

In [19]:
 y_train.value_counts(normalize=True)

1    0.503237
0    0.496763
Name: toxic, dtype: float64

Для того, чтобы провести кросс-валидацию KFold сбросим индексы

In [39]:
X_train=X_train.reset_index(drop=True)

In [40]:
y_train=y_train.reset_index(drop=True)

In [41]:
from sklearn.model_selection import KFold

In [42]:
kf=KFold(n_splits=3)

In [43]:
for train_index,test_index in kf.split(X_train):
    print(train_index)

[ 7465  7466  7467 ... 22392 22393 22394]
[    0     1     2 ... 22392 22393 22394]
[    0     1     2 ... 14927 14928 14929]


При проведении кросс-валидации будем использовать только LogisticRegression, она лучше всех показала себя в предыдущей CV.

In [44]:
for train_index,test_index in kf.split(X_train):
    results=[]
    X_train_cv=X_train[train_index]
    X_test_cv=X_train[test_index]
    y_train_cv=y_train[train_index]
    y_test_cv=y_train[test_index]
    print()
    train_embeddings=BERT(X_train_cv)
    test_embeddings=BERT(X_test_cv)
    model_lr=LogisticRegression()
    model_lr.fit(train_embeddings,y_train_cv.iloc[:len(train_embeddings)])
    predicted=model_lr.predict(test_embeddings)
    print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test_cv.iloc[:len(test_embeddings)], predicted))
    print("Logistic Regression Precision:",metrics.precision_score(y_test_cv.iloc[:len(test_embeddings)], predicted))
    print("Logistic Regression Recall:",metrics.recall_score(y_test_cv.iloc[:len(test_embeddings)], predicted))
    print("Logistic Regression F1:",metrics.f1_score(y_test_cv.iloc[:len(test_embeddings)], predicted))
    results+=metrics.f1_score(y_test_cv.iloc[:len(test_embeddings)], predicted)


Logistic Regression Accuracy: 0.8794630872483221
Logistic Regression Precision: 0.8919067215363512
Logistic Regression Recall: 0.8657789613848202
Logistic Regression F1: 0.8786486486486487



C:\Users\Andrey\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)



Logistic Regression Accuracy: 0.881744966442953
Logistic Regression Precision: 0.8868852459016393
Logistic Regression Recall: 0.8742256935092917
Logistic Regression F1: 0.8805099688050997



C:\Users\Andrey\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)



Logistic Regression Accuracy: 0.8769127516778523
Logistic Regression Precision: 0.8859530870854678
Logistic Regression Recall: 0.8693121693121693
Logistic Regression F1: 0.8775537454933904


C:\Users\Andrey\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


### Вывод

- мы выборали модель BERT, загрузили ее и токенизатор.
- токенизировали данные, ращделили выборку на тестовую и обучающую
- на обучающей выборке мы провели кросс-валидацию модели LogisticRegression
- результат F1 меры оказался равен 0.88

### Test

In [46]:
X_test_emb=BERT(X_test)

In [62]:
X_train_emb=BERT(X_train)

In [63]:
model_lr.fit(X_train_emb,y_train.iloc[:len(X_train_emb)])

C:\Users\Andrey\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [64]:
prediction=model_lr.predict(X_test_emb)

In [65]:
print("Logistic Regression F1:",metrics.f1_score(y_test.iloc[:len(X_test_emb)], prediction))
    

Logistic Regression F1: 0.884928402183991


### Вывод

- мы протестировали нашу модель
- результат оказался 0.88, что выше целевого 0.75

# 3. Выводы

- Мы изучили данные
1. Ипользование SPACY
    - Провели лемматизацию
    - Провели кросс-валидацию
    - Лучшей моделью по ее резльтатам оказалась LogisticRegression с результатом 0.75
    - Провели тестирование на тестовой выборке, результат составил 0.75
2. Использование BERT
    - мы выборали модель BERT, загрузили ее и токенизатор.
    - токенизировали данные, раpделили выборку на тестовую и обучающую
    - на обучающей выборке мы провели кросс-валидацию модели LogisticRegression
    - результат F1 меры оказался равен 0.88
    - результат F! меры на тестовой выборке оказался равен 0.88
____
Модель BERT показала лучшие результаты по сравнению с SPACY и tf-idf методом.  
Но трудозатраты и время обучения у первой сильно выше, хотя в обоих слуаях мы смогли достичь требуемое качество целевой метрики